In [ ]:
! pip install "autogen-ext[openai]"
! pip install autogen-agentchat
! pip install azure-ai-projects 
! pip install azure-identity

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool,CodeInterpreterTool

In [ ]:
import os

In [ ]:
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

In [ ]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4",
    api_version="2024-12-01-preview",
    model = "gpt-4",
    azure_endpoint="our Azure OpenAI Service endpoint from Azure AI Foundry",
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key="Your API Key from Azure AI Foundry", # For key-based authentication.
)

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Project connection string from Azure Foundry here',
)

In [ ]:
bing_connection = project_client.connections.get(
    connection_name='Grounding with Bing Search connection name from Azure Foundry here'
)

conn_id = bing_connection.id

In [ ]:
async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4",
            name="my-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        # project_client.agents.delete_agent(agent.id)
        # print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        # print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [ ]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")
    print(blog_content)
    with project_client:

        code_interpreter = CodeInterpreterTool()
        
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="my-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
        )
        
        thread = project_client.agents.create_thread()

        message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content="""
                    You are my Python programming assistant. Generate code and execute it according to the following requirements

                    1. Save """ + blog_content + """ to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    
        )
        # create and execute a run
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
            print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
        messages = project_client.agents.list_messages(thread_id=thread.id)
        # print(f"Messages: {messages}")

        # get the most recent message from the assistant
        # last_msg = messages["data"][0]["content"][0]["text"]["value"]
        # if last_msg:
        #   print(f"Last Message: {last_msg}")

        # print(f"File: {messages.file_path_annotations}")


        for file_path_annotation in messages.file_path_annotations:

            file_name = os.path.basename(file_path_annotation.text)

            project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
            

        #project_client.agents.delete_agent(agent.id)
        #print("Deleted agent")
        return "Saved"


    
    

In [ ]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

In [ ]:
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)

In [ ]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)


In [ ]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [ ]:

reflection_team = RoundRobinGroupChat([write_agent, save_blog_content_agent], termination_condition=termination)

In [ ]:
await Console(
                reflection_team.run_stream(task="""

                   I am writing a blog about machine learning. Search for the following 3 questions and write a  blog based on the search results ,save it
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning

                               

    """)
)  # Stream the messages to the console.